# Prototype notebook to semantically represent Learning Path with Schema.org

authors: Phil Reed, Alban Gaignard

drafted the 20th of November 2025, as part of Bioschemas activities

To be revised by the community.

Any questions, please contact phil.reed@manchester.ac.uk, alban.gaignard@univ-nantes.fr



In [1]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.0/569.0 kB 27.8 MB/s eta 0:00:00


# Sample Learning path
https://tess.elixir-europe.org/learning_paths/introduction-to-galaxy-and-sequence-analysis-6384c0ed-3546-41cf-ac30-bff8680dd96c

LP structure :
- Module 1
  - TrainingMaterial 11
  - TrainingMaterial 12 (after TM11)
- Module 2 (after module 1)
  - TrainingMaterial 21
  - TrainingMaterial 22 (after TM21)
  - TrainingMaterial 23 (after TM22)
  - TrainingMaterial 24 (after TM23)

In [18]:
from rdflib import Graph

my_learning_path = """
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .
@prefix ex: <http://example.org/> .

ex:GA_learning_path a schema:Course, schema:HowTo ;
    schema:name "Introduction to Galaxy and Sequence analysis" ;
    schema:description "A foundational course for Galaxy and Sequence analysis." ;
    schema:provider ex:ExampleUniversity ;
    schema:courseCode "GSA101" ;
    schema:step ex:Module_2, ex:Module_1 .

ex:Module_1 a schema:HowToSection, schema:Course ;
    schema:name "Module 1" ;
    schema:nextItem ex:Module_2 ;
    schema:itemListElement ex:TM11, ex:TM12, ex:TM3 .

ex:Module_2 a schema:HowToSection, schema:Course ;
    schema:name "Module 2" ;
    schema:itemListElement ex:TM21, ex:TM22, ex:TM23, ex:TM24, ex:TM3 .


ex:TM11 a schema:LearningResource, schema:HowToStep ;
    schema:name "TrainingMaterial 11" ;
    schema:description "Description of TrainingMaterial 11" ;
    schema:url "https://tess.elixir-europe.org/materials/hands-on-for-a-short-introduction-to-galaxy-tutorial?lp=1%3A1" ;
    schema:nextItem ex:TM12 .

ex:TM12 a schema:LearningResource, schema:HowToStep ;
    schema:name "TrainingMaterial 12" ;
    schema:description "Description of TrainingMaterial 12" ;
    schema:nextItem ex:Module_2, ex:TM21, ex:TM3 .

ex:TM21 a schema:LearningResource, schema:HowToStep ;
    schema:name "TrainingMaterial 21" ;
    schema:description "Description of TrainingMaterial 21" ;
    schema:nextItem ex:TM22 .

ex:TM22 a schema:LearningResource, schema:HowToStep ;
    schema:name "TrainingMaterial 22" ;
    schema:description "Description of TrainingMaterial 22" ;
    schema:nextItem ex:TM23 .

ex:TM23 a schema:LearningResource, schema:HowToStep ;
    schema:name "TrainingMaterial 23" ;
    schema:description "Description of TrainingMaterial 23" ;
    schema:nextItem ex:TM24 .

ex:TM24 a schema:LearningResource, schema:HowToStep ;
    schema:name "TrainingMaterial 24" ;
    schema:description "Description of TrainingMaterial 24" ;
    schema:nextItem ex:TM3 .

ex:TM3 a schema:LearningResource, schema:HowToStep ;
    schema:name "TrainingMaterial shared by Module 1 and 2" .
"""

kg = Graph()
kg.parse(data=my_learning_path, format="turtle")

<Graph identifier=N0b5207e042ae4f0fb091399e4e3dd407 (<class 'rdflib.graph.Graph'>)>

In [19]:
print(kg.serialize(format="turtle"))
print(len(kg))

@prefix ex: <http://example.org/> .
@prefix schema: <https://schema.org/> .

ex:GA_learning_path a schema:Course,
        schema:HowTo ;
    schema:courseCode "GSA101" ;
    schema:description "A foundational course for Galaxy and Sequence analysis." ;
    schema:name "Introduction to Galaxy and Sequence analysis" ;
    schema:provider ex:ExampleUniversity ;
    schema:step ex:Module_1,
        ex:Module_2 .

ex:Module_1 a schema:Course,
        schema:HowToSection ;
    schema:itemListElement ex:TM11,
        ex:TM12,
        ex:TM3 ;
    schema:name "Module 1" ;
    schema:nextItem ex:Module_2 .

ex:TM11 a schema:HowToStep,
        schema:LearningResource ;
    schema:description "Description of TrainingMaterial 11" ;
    schema:name "TrainingMaterial 11" ;
    schema:nextItem ex:TM12 ;
    schema:url "https://tess.elixir-europe.org/materials/hands-on-for-a-short-introduction-to-galaxy-tutorial?lp=1%3A1" .

ex:TM12 a schema:HowToStep,
        schema:LearningResource ;
    schema:desc

In [20]:
print(kg.serialize(format="json-ld"))

[
  {
    "@id": "http://example.org/Module_2",
    "@type": [
      "https://schema.org/HowToSection",
      "https://schema.org/Course"
    ],
    "https://schema.org/itemListElement": [
      {
        "@id": "http://example.org/TM21"
      },
      {
        "@id": "http://example.org/TM22"
      },
      {
        "@id": "http://example.org/TM23"
      },
      {
        "@id": "http://example.org/TM24"
      },
      {
        "@id": "http://example.org/TM3"
      }
    ],
    "https://schema.org/name": [
      {
        "@value": "Module 2"
      }
    ]
  },
  {
    "@id": "http://example.org/TM23",
    "@type": [
      "https://schema.org/LearningResource",
      "https://schema.org/HowToStep"
    ],
    "https://schema.org/description": [
      {
        "@value": "Description of TrainingMaterial 23"
      }
    ],
    "https://schema.org/name": [
      {
        "@value": "TrainingMaterial 23"
      }
    ],
    "https://schema.org/nextItem": [
      {
        "@id": "http:/

In [34]:
# I'm interested in a specific training material, what should be the prerequisites ?

tm_of_interest = "TrainingMaterial 21"

def query_learning_path():
  query = f"""
  SELECT * WHERE {{
    ?s schema:name "{tm_of_interest}" .
    ?prereq schema:nextItem+ ?s .
    ?prereq schema:name ?prereq_name .
    ?a_module schema:itemListElement ?prereq .
    ?a_module schema:name ?module_name .
  }}
  """
  # Run SPARQL query
  res_kg = kg.query(query)
  print(f'If you are interested in {tm_of_interest}')
  for r in res_kg:
    print(f'You need to perform {r["prereq_name"]} as part of the module {r["module_name"]}')
  return res_kg

query_learning_path()

If you are interested in TrainingMaterial 21
You need to perform TrainingMaterial 12 as part of the module Module 1
You need to perform TrainingMaterial 11 as part of the module Module 1


## What we did until now

1.  **RDF Graph Initialization**: We loaded a learning path described in Turtle format into an RDFLib graph (`kg`). We extensively used Schema.org to model the LearningPath HowTo, HowToSection and HowToStep types, as well as Course and LearningResource types.
2.  **Prerequisite Query**: We demonstrated how to query the graph using SPARQL to find prerequisites for a specific learning material.


# Visualization of the Schema.org metadata

In [35]:
from rdflib import Graph, Literal, URIRef
from rdflib.namespace import RDF, RDFS, OWL, XSD

def extract_relationships_and_names(graph):
    # Define prefixes
    schema = URIRef("https://schema.org/")
    ex = URIRef("http://example.org/")

    # 1. Query for all entities with a schema:name and store their names
    entity_names = {}
    name_query = """
    PREFIX schema: <https://schema.org/>
    SELECT ?entity ?name
    WHERE {
        ?entity schema:name ?name .
    }
    """
    for row in graph.query(name_query):
        entity_names[str(row.entity)] = str(row.name)

    print("Extracted entity names:")
    for uri, name in entity_names.items():
        print(f"  {uri}: {name}")

    # 2. Query for sequential relationships
    relationships_query = """
    PREFIX schema: <https://schema.org/>
    SELECT ?s ?p ?o
    WHERE {
        ?s ?p ?o .
        FILTER (?p = schema:nextItem || ?p = schema:itemListElement || ?p = schema:step)
    }
    """
    raw_relationships = []
    for row in graph.query(relationships_query):
        raw_relationships.append((str(row.s), str(row.p), str(row.o)))

    print("\nExtracted raw relationships:")
    for rel in raw_relationships:
        print(f"  {rel[0]} -- {rel[1]} --> {rel[2]}")

    # 3. Transform relationships to use names
    mermaid_edges = []
    for s_uri, p_uri, o_uri in raw_relationships:
        s_name = entity_names.get(s_uri, s_uri.split('/')[-1]) # Use URI fragment if name not found
        o_name = entity_names.get(o_uri, o_uri.split('/')[-1]) # Use URI fragment if name not found
        predicate_type = p_uri.split('/')[-1] # Get predicate name (e.g., nextItem)
        mermaid_edges.append((s_name, predicate_type, o_name))

    print("\nTransformed relationships for Mermaid diagram:")
    for edge in mermaid_edges:
        print(f"  {edge[0]} -- {edge[1]} --> {edge[2]}")

    return entity_names, mermaid_edges

# Call the function with the existing kg graph
all_entity_names, all_mermaid_edges = extract_relationships_and_names(kg)

# Assign to the global query_learning_path function as well if needed later
def query_learning_path():
    return all_entity_names, all_mermaid_edges

print("Relationship extraction complete. Results stored in all_entity_names and all_mermaid_edges.")

Extracted entity names:
  http://example.org/GA_learning_path: Introduction to Galaxy and Sequence analysis
  http://example.org/Module_1: Module 1
  http://example.org/Module_2: Module 2
  http://example.org/TM11: TrainingMaterial 11
  http://example.org/TM12: TrainingMaterial 12
  http://example.org/TM21: TrainingMaterial 21
  http://example.org/TM22: TrainingMaterial 22
  http://example.org/TM23: TrainingMaterial 23
  http://example.org/TM24: TrainingMaterial 24
  http://example.org/TM3: TrainingMaterial shared by Module 1 and 2

Extracted raw relationships:
  http://example.org/Module_1 -- https://schema.org/itemListElement --> http://example.org/TM3
  http://example.org/Module_2 -- https://schema.org/itemListElement --> http://example.org/TM21
  http://example.org/Module_2 -- https://schema.org/itemListElement --> http://example.org/TM24
  http://example.org/TM12 -- https://schema.org/nextItem --> http://example.org/TM3
  http://example.org/GA_learning_path -- https://schema.org/s

In [37]:
mermaid_lines = []
mermaid_lines.append('graph TD')

for s_name, p_type, o_name in all_mermaid_edges:
    # Sanitize names for Mermaid if they contain special characters or spaces
    # Mermaid node IDs cannot contain spaces or special characters directly
    # We'll use a simple approach: if a name has spaces, enclose it in quotes.
    # For simplicity, we'll map original names to safe IDs and use names for labels

    # Create a unique ID for each node if it's not already in the entity_names (which are already unique names)
    # For this exercise, we will assume the names are unique enough for node IDs or simply use them as both ID and label.
    # Mermaid allows IDs with spaces if quoted, but it's cleaner to have simple IDs.
    # However, given the prompt directly asks for f'{source_name} -- {predicate_type} --> {target_name}',
    # we will use the names as they are, and Mermaid should handle simple spaces correctly in labels.

    # Let's ensure node labels are quoted if they contain spaces to be robust
    # Mermaid syntax: A["Node Name"] or A --> B["Another Node"]
    # For simplicity, let's use the names directly as node IDs and labels.
    # Mermaid handles spaces in node labels gracefully if they are quoted, but for connections
    # the IDs should be simple. Let's make sure our approach is compatible.

    # A common Mermaid practice is to have a simple ID for the node and then a label
    # like: ID[Label]. If we use the name as ID, it should not have spaces.
    # Given the previous step produced names like 'Module 1', we should create a mapping for node IDs.

    # Let's refine how we generate the Mermaid lines to ensure valid syntax
    # Create a mapping from clean name to a unique, mermaid-safe ID
    # This mapping is important to handle cases where names might have characters that Mermaid IDs don't like
    node_id_map = {}
    counter = 0
    for uri_name in all_entity_names.values():
        clean_name = uri_name.replace(' ', '_').replace('-', '_').replace(':', '_').replace('.', '_').strip()
        if clean_name not in node_id_map.values():
            node_id_map[uri_name] = f'N{counter}'
            counter += 1

    # Create mermaid lines with unique IDs and original names as labels
    mermaid_lines = ['graph TD']
    for s_name, p_type, o_name in all_mermaid_edges:
        source_id = node_id_map.get(s_name, s_name.replace(' ', '_').replace('-', '_').replace(':', '_').replace('.', '_').strip())
        target_id = node_id_map.get(o_name, o_name.replace(' ', '_').replace('-', '_').replace(':', '_').replace('.', '_').strip())

        # Add node definitions with labels if they contain spaces
        if ' ' in s_name:
            mermaid_lines.append(f'{source_id}[\"{s_name}\"]')
        if ' ' in o_name:
            mermaid_lines.append(f'{target_id}[\"{o_name}\"]')

        mermaid_lines.append(f'{source_id} -- {p_type} --> {target_id}')

mermaid_syntax = '\n'.join(mermaid_lines)

# display the markdown mermaid graph as output of this cell
display(mermaid_syntax)

'graph TD\nN1["Module 1"]\nN9["TrainingMaterial shared by Module 1 and 2"]\nN1 -- itemListElement --> N9\nN2["Module 2"]\nN5["TrainingMaterial 21"]\nN2 -- itemListElement --> N5\nN2["Module 2"]\nN8["TrainingMaterial 24"]\nN2 -- itemListElement --> N8\nN4["TrainingMaterial 12"]\nN9["TrainingMaterial shared by Module 1 and 2"]\nN4 -- nextItem --> N9\nN0["Introduction to Galaxy and Sequence analysis"]\nN2["Module 2"]\nN0 -- step --> N2\nN4["TrainingMaterial 12"]\nN2["Module 2"]\nN4 -- nextItem --> N2\nN1["Module 1"]\nN2["Module 2"]\nN1 -- nextItem --> N2\nN7["TrainingMaterial 23"]\nN8["TrainingMaterial 24"]\nN7 -- nextItem --> N8\nN5["TrainingMaterial 21"]\nN6["TrainingMaterial 22"]\nN5 -- nextItem --> N6\nN2["Module 2"]\nN6["TrainingMaterial 22"]\nN2 -- itemListElement --> N6\nN3["TrainingMaterial 11"]\nN4["TrainingMaterial 12"]\nN3 -- nextItem --> N4\nN4["TrainingMaterial 12"]\nN5["TrainingMaterial 21"]\nN4 -- nextItem --> N5\nN2["Module 2"]\nN7["TrainingMaterial 23"]\nN2 -- itemListEle

In [40]:
from IPython.display import display, Markdown
Markdown(mermaid_syntax)

graph TD
N1["Module 1"]
N9["TrainingMaterial shared by Module 1 and 2"]
N1 -- itemListElement --> N9
N2["Module 2"]
N5["TrainingMaterial 21"]
N2 -- itemListElement --> N5
N2["Module 2"]
N8["TrainingMaterial 24"]
N2 -- itemListElement --> N8
N4["TrainingMaterial 12"]
N9["TrainingMaterial shared by Module 1 and 2"]
N4 -- nextItem --> N9
N0["Introduction to Galaxy and Sequence analysis"]
N2["Module 2"]
N0 -- step --> N2
N4["TrainingMaterial 12"]
N2["Module 2"]
N4 -- nextItem --> N2
N1["Module 1"]
N2["Module 2"]
N1 -- nextItem --> N2
N7["TrainingMaterial 23"]
N8["TrainingMaterial 24"]
N7 -- nextItem --> N8
N5["TrainingMaterial 21"]
N6["TrainingMaterial 22"]
N5 -- nextItem --> N6
N2["Module 2"]
N6["TrainingMaterial 22"]
N2 -- itemListElement --> N6
N3["TrainingMaterial 11"]
N4["TrainingMaterial 12"]
N3 -- nextItem --> N4
N4["TrainingMaterial 12"]
N5["TrainingMaterial 21"]
N4 -- nextItem --> N5
N2["Module 2"]
N7["TrainingMaterial 23"]
N2 -- itemListElement --> N7
N8["TrainingMaterial 24"]
N9["TrainingMaterial shared by Module 1 and 2"]
N8 -- nextItem --> N9
N6["TrainingMaterial 22"]
N7["TrainingMaterial 23"]
N6 -- nextItem --> N7
N2["Module 2"]
N9["TrainingMaterial shared by Module 1 and 2"]
N2 -- itemListElement --> N9
N1["Module 1"]
N4["TrainingMaterial 12"]
N1 -- itemListElement --> N4
N1["Module 1"]
N3["TrainingMaterial 11"]
N1 -- itemListElement --> N3
N0["Introduction to Galaxy and Sequence analysis"]
N1["Module 1"]
N0 -- step --> N1